In [30]:
import numpy as np
# the layer number of each stage
stage_layer_max = [2, 4, 14, 2]
# the channel number of each layer
layer = np.array([64, 128, 256, 512])
# the descending step of channel 
step = 8 
op = ['repvgg','vgg'] # every stage

maxpool = ['T','F']
ratio = [0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0]
def search_space(stage_layer, layer):
    sum = 1
    for i in stage_layer_max:
        sum *= i
    for j in layer:
        sum *= len(ratio)
    return sum*16*16
print(search_space(stage_layer_max,layer)/1e8)

2.34881024


In [38]:
from random import choices 
choices(range(1,3))[0]

2

In [44]:
choices(ratio)[0]

0.875

In [46]:
# 随机生成模型

stage_layer = [choices(range(1,stage_layer_max[0]+1))[0],choices(range(1,stage_layer_max[1]+1))[0],choices(range(1,stage_layer_max[2]+1))[0],choices(range(1,stage_layer_max[3]+1))[0]]
# stage_layer
stage_ratio = [choices(ratio)[0],choices(ratio)[0],choices(ratio)[0],choices(ratio)[0]]
# stage_ratio
with_maxpool = [choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0],choices(maxpool)[0]]
with_maxpool

['F', 'T', 'F', 'F']

In [ ]:
import yaml
def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))
 
def read_yaml_to_dict(yaml_path: str):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value
config = {}
dir = 
config['model'] = {'stage_layer':stage_layer,'stage_ratio': stage_ratio ,'with_maxpool':with_maxpool}
config['train'] = {'start_epoch': 0, "epochs": 300, "warmup_epochs": 20, "warmup_lr": 1.0e-3, 
                   "lr_scheduler":{"min_lr":1.0e-5,"decay_epochs":30,"decay_rate":0.1},"batch_size":256,"loss":"crossentropy",
                   "optimizer":{"name":"SGD","base_lr":0.0005,"momentum":0.9,"eps":1.0e-8,"betas":[0.9,0.999],"weight_decay_param":{"decay":0.05,"echo":False}},
                   "label_smoothing":0.1,"clip_grad":0.0,"ema_alpha":0.0,"ema_update_period":8,"use_l2_norm":True,
                   "no_weight_decay": "rbr_dense"}
config['val'] = {"batch_size":128} 
config['output'] = {"print_freq":100,"epoch_print_freq":1,"save_freq":20,
                    "dir": }


In [ ]:
# 训练自动生成yaml
# loss acc train val
# 存test最好的
# loss crossentropy
# SGD
# lr 0.1，0.001（repvgg
# weight decay 0.005，0.05（repvgg
# 直接量化repvgg， repvgg block 的量化，需要考虑lr和weight decay

# 随机生成模型配置，训
# 得到小数据集
# acc predictor， 是配置 （nas + predictor
# 搜索算法 RL EL （chip memory 约束项，要提acc，降latency
# 硬件性能 
# hardware performance 
# 硬件结构固定，公式计算硬件性能
# few-shot learning
# meta-learning
# 搜索考虑memory

# 1. search space
# 2. 重点是predictor
# 3. 

# random genea
# 进一步， 搜索对不同硬件都高效的网络
# 



In [24]:
24/64

0.375

In [2]:
2*4*14*2*31*63*127*255

14167530720

In [10]:
import torch
import torch.nn as nn
import torch.utils.checkpoint as checkpoint
import numpy as np

/home/qhy/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups=1):
    result = nn.Sequential()
    result.add_module('conv', nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups, bias=False))
    result.add_module('bn', nn.BatchNorm2d(num_features=out_channels))
    return result

In [15]:
class RepVGGBlock(nn.Module):
    """
        Naive RepVGG without SE module
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, padding_mode='zeros', deploy=False):
        super(RepVGGBlock, self).__init__()
        self.deploy = deploy
        self.groups = groups
        self.in_channels = in_channels

        self.non_linearity = nn.ReLU()

        assert kernel_size == 3
        assert padding == 1

        padding_11 = padding - kernel_size // 2

        if deploy:
            self.rbr_reparam = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=True, padding_mode=padding_mode)
        else:
            self.rbr_identity = nn.BatchNorm2d(num_features=out_channels) if out_channels == in_channels and stride == 1 else None
            self.rbr_dense = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, groups=groups)
            self.rbr_1x1 = conv_bn(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=stride, padding=padding_11, groups=groups)
    
    def forward(self, inputs):
        if hasattr(self, 'rbr_reparam'):
            return self.non_linearity(self.rbr_reparam(inputs))
        
        if self.rbr_identity is None:
            id_out = 0
        else:
            id_out = self.rbr_identity(inputs)
        
        return self.non_linearity(self.rbr_dense(inputs) + self.rbr_1x1(inputs) + id_out)
    
    def get_custom_L2(self):
        K3 = self.rbr_dense.conv.weight
        K1 = self.rbr_1x1.conv.weight
        t3 = (self.rbr_dense.bn.weight / ((self.rbr_dense.bn.running_var + self.rbr_dense.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()
        t1 = (self.rbr_1x1.bn.weight / ((self.rbr_1x1.bn.running_var + self.rbr_1x1.bn.eps).sqrt())).reshape(-1, 1, 1, 1).detach()

        l2_loss_circle = (K3 ** 2).sum() - (K3[:, :, 1:2, 1:2] ** 2).sum()      # The L2 loss of the "circle" of weights in 3x3 kernel. Use regular L2 on them.
        eq_kernel = K3[:, :, 1:2, 1:2] * t3 + K1 * t1                           # The equivalent resultant central point of 3x3 kernel.
        l2_loss_eq_kernel = (eq_kernel ** 2 / (t3 ** 2 + t1 ** 2)).sum()        # Normalize for an L2 coefficient comparable to regular L2.
        return l2_loss_eq_kernel + l2_loss_circle

    def get_equivalent_kernel_bias(self):
        kernel3x3, bias3x3 = self._fuse_bn_tensor(self.rbr_dense)
        kernel1x1, bias1x1 = self._fuse_bn_tensor(self.rbr_1x1)
        kernelid, biasid = self._fuse_bn_tensor(self.rbr_identity)
        return kernel3x3 + self._pad_1x1_to_3x3_tensor(kernel1x1) + kernelid, bias3x3 + bias1x1 + biasid

    def _pad_1x1_to_3x3_tensor(self, kernel1x1):
        if kernel1x1 is None:
            return 0
        else:
            return torch.nn.functional.pad(kernel1x1, [1, 1, 1, 1])
    
    def _fuse_bn_tensor(self, branch):
        if branch is None:
            return 0, 0
        if isinstance(branch, nn.Sequential):
            kernel = branch.conv.weight
            running_mean = branch.bn.running_mean
            running_var = branch.bn.running_var
            gamma = branch.bn.weight
            beta = branch.bn.bias
            eps = branch.bn.eps
        else:
            assert isinstance(branch, nn.BatchNorm2d)
            if not hasattr(self, 'id_tensor'):
                input_dim = self.in_channels // self.groups
                kernel_value = np.zeros((self.in_channels, input_dim, 3, 3), dtype=np.float32)
                for i in range(self.in_channels):
                    kernel_value[i, i % input_dim, 1, 1] = 1
                self.id_tensor = torch.from_numpy(kernel_value).to(branch.weight.device)
            kernel = self.id_tensor
            running_mean = branch.running_mean
            running_var = branch.running_var
            gamma = branch.weight
            beta = branch.bias
            eps = branch.eps
        std = (running_var + eps).sqrt()
        t = (gamma / std).reshape(-1, 1, 1, 1)
        return kernel * t, beta - running_mean * gamma / std
    
    def switch_to_deploy(self):
        if hasattr(self, 'rbr_reparam'):
            return
        kernel, bias = self.get_equivalent_kernel_bias()
        self.rbr_reparam = nn.Conv2d(in_channels=self.rbr_dense.conv.in_channels, out_channels=self.rbr_dense.conv.out_channels, kernel_size=self.rbr_dense.conv.kernel_size, stride=self.rbr_dense.conv.stride, padding=self.rbr_dense.conv.padding, dilation=self.rbr_dense.conv.dilation, groups=self.rbr_dense.conv.groups, bias=True)
        self.rbr_reparam.weight.data = kernel
        self.rbr_reparam.bias.data = bias
        self.__delattr__('rbr_dense')
        self.__delattr__('rbr_1x1')
        if hasattr(self, 'rbr_identity'):
            self.__delattr__('rbr_identity')
        if hasattr(self, 'id_tensor'):
            self.__delattr__('id_tensor')
        self.deploy = True

In [31]:
RepVGGBlock(3,64,3,padding=1)

RepVGGBlock(
  (non_linearity): ReLU()
  (rbr_dense): Sequential(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (rbr_1x1): Sequential(
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [ ]:
# padding == 1 是为了保证尺寸不变


In [22]:
cfgs = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}
def make_layers(cfg, batch_norm=False):
    '''
    根据配置表，返回模型层列表
    '''
    layers = [] # 层列表初始化

    in_channels = 3 # 输入3通道图像

    # 遍历配置列表
    for v in cfg:
        if v == 'M': # 添加池化层
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else: # 添加卷积层

            # 3×3 卷积
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)

            # 卷积-->批归一化（可选）--> ReLU激活
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]

            # 通道数方面，下一层输入即为本层输出
            in_channels = v

    # 以sequencial类型返回模型层列表
    return nn.Sequential(*layers)

# make_layers(cfgs['A'], batch_norm=True)

In [21]:
class VGG(nn.Module):
    '''
    VGG通用网络模型
    输入features为网络的特征提取部分网络层列表
    分类数为 1000
    '''
    def __init__(self, features, dataset = 'cifar10', init_weights=True):
        super(VGG, self).__init__()
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
            
        # 特征提取部分
        self.features = features

        # 自适应平均池化，特征图池化到 7×7 大小
        # 为了让下一步的flatten得到一个固定长度的向量
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))

        # 分类部分
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),   # 512*7*7 --> 4096
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),          # 4096 --> 4096
            nn.ReLU(True),
            # nn.Dropout(),
            nn.Linear(4096, num_classes),   # 4096 --> 1000
        )

        # 权重初始化
        if init_weights:
            self._initialize_weights()

    def forward(self, x):

        # 特征提取
        x = self.features(x)
        # 自适应平均池化
        x = self.avgpool(x)
        # 特征图展平成向量
        x = torch.flatten(x, 1)
        # 分类器分类输出
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        '''
        权重初始化
        '''
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 卷积层使用 kaimming 初始化
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
                # 偏置初始化为0
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            # 批归一化层权重初始化为1 
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            # 全连接层权重初始化
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [ ]:

model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)

